### Check for files and initialise download

In [26]:
import os
import subprocess

# Configuration
em_code = "bc013"  # Example EM code, replace with your actual value
base_path = "/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/"
remote_user = "masher92"
remote_host = "xfer1.jasmin.ac.uk"
remote_base_dir = "/gws/nopw/j04/icasp_swf/masher/"


# Function to check file existence and download if absent
def check_and_download(year, month, day):
    file_pattern = f"{em_code}a.pr{year}{month}{day}.pp"
    local_file_path = os.path.join(base_path, em_code, file_pattern)
    remote_file_path = os.path.join(remote_base_dir, em_code, file_pattern)

    # Check if the file already exists
    if not os.path.exists(local_file_path):
        #print(f"{file_pattern}")
        print(f'scp masher92@xfer1.jasmin.ac.uk:/gws/nopw/j04/icasp_swf/masher/{em_code}/{file_pattern} "/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/{em_code}/"')
        #print(f"pp_file={file_pattern}")
        
        # Construct the scp command
        scp_command = [
            "scp",
            f"{remote_user}@{remote_host}:{remote_file_path}",
            local_file_path
        ]
        # Call the scp command
        try:
            subprocess.run(scp_command, check=True)
            print(f"Successfully downloaded {file_pattern}")
        except subprocess.CalledProcessError:
            print(f"Failed to download {file_pattern}")
    else:
        #print(f"File {file_pattern} already exists, skipping...")
        pass


# Example usage for a specific year, month, and day
for year in range(2001,2021):
    for month in range(1,13):
        month_str = f"{month:02d}"
        for day in range(1, 31):  # Assuming you start from 16th to 30th
            day_str = f"{day:02d}"  # Zero-padding the day
            
            check_and_download(year, month_str, day_str)


scp masher92@xfer1.jasmin.ac.uk:/gws/nopw/j04/icasp_swf/masher/bc013/bc013a.pr20180209.pp "/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2002_2020/bc013/"



            Access to this system is monitored and restricted to
            authorised users.   If you do not have authorisation
            to use  this system,  you should not  proceed beyond
            this point and should disconnect immediately.

            Unauthorised use could lead to prosecution.

    (See also - http://www.stfc.ac.uk/aup)



Successfully downloaded bc013a.pr20180209.pp


### Check for files 

In [11]:
# for em_1hr in ['01', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '15']:
for em_1hr in ['12']:
    em_30mins = em_matching_dict[em_1hr]
    for yr in range(2001,2021):
        for month_num in ['01', '02', '03', '04', '05', '09', '10','11','12']:
            print(f"em{em_1hr}, yr {yr}, month {month_num}")
            if (os.path.isfile(f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/{resolution}/{em_30mins}/{yrs_range}/{em_30mins}a.pr{yr}{month_num}.nc")):
                print("already exists")
            else:
                print(f"Running for month {month_num} in {yr}, for {em_1hr} (which equatees to {em_30mins})")

                ####################################################### 
                #######################################################
                ## Get one month of data - HOURLY
                ####################################################### 
                #######################################################
                ### Get a list of filenames for hourly data
                general_filename_1hr = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/{em_1hr}/{yrs_range}/pr_rcp85_land-cpm_uk_2.2km_{em_1hr}_1hr_{yr}{month_num}*'
                filenames_1hr = []
                for filename in glob.glob(general_filename_1hr):
                        filenames_1hr.append(filename)
                # If don't find any files matching this string in the 2001_2020 folder, then check the 1980_2001
                if len(filenames_1hr) == 0:
                    general_filename_1hr = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/{em_1hr}/1980_2001/pr_rcp85_land-cpm_uk_2.2km_{em_1hr}_1hr_{yr}{month_num}*'
                    for filename in glob.glob(general_filename_1hr):
                            filenames_1hr.append(filename)
                    print(len(filenames_1hr))

                # ### Load in the data and remove the ensemble member dimension
                monthly_cubes_list_1hr = iris.load(filenames_1hr)
                cube_1hr = monthly_cubes_list_1hr[0]
                cube_1hr = cube_1hr[0,:,:,:]

                # ### Trim to Leeds
                # cube_1hr = trim_to_bbox_of_region(cube_1hr, leeds_at_centre_gdf)

                #######################################################
                #######################################################
                ## Get one month of data - 30mins
                #######################################################
                #######################################################
                # ### Get all files for this ensemble member
                general_filename_30mins = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/{yrs_range}/{em_30mins}/{em_30mins}a.pr{yr}{month_num}*'
                filenames_first30mins = []
                for filename_30mins in glob.glob(general_filename_30mins):
                    filenames_first30mins.append(filename_30mins)
                filenames_first30mins.sort()

                # ### Load in the data 
                monthly_cubes_list_30mins = iris.load(filenames_first30mins)

                # Equalise
                for cube in monthly_cubes_list_30mins:
                    for attr in ['forecast_period', 'forecast_reference_time']:
                        if attr in cube.attributes:
                            del cube.attributes[attr]


                monthly_cube_30mins = monthly_cubes_list_30mins.concatenate_cube()      

                # ### Trim to be the same shape as the hourly data
                # monthly_cube_30mins_1st = trim_to_bbox_of_region_30mins(monthly_cube_30mins, leeds_at_centre_gdf)
                monthly_cube_30mins_1st = monthly_cube_30mins[:,24:-24,24:-24]

                # ### Convert units of 30 mins data
                # Check current units
                # print(monthly_cube_30mins_1st.units)
                # Set the units to those of the 1 hr cube
                monthly_cube_30mins_1st.units = cube_1hr.units
                # print(monthly_cube_30mins_1st.units)

                # Convert the data to also be this unit
                monthly_cube_30mins_1st_data = monthly_cube_30mins_1st.data
                monthly_cube_30mins_1st_data = monthly_cube_30mins_1st_data*3600

                monthly_cube_30mins_1st.data = monthly_cube_30mins_1st_data


                #######################################################
                #######################################################
                ## Find the second half of the hour, using the first half of hour and hourly values
                #######################################################
                #######################################################
                # get the hourly data
                cube_1hr_data = cube_1hr.data
                
                if cube_1hr_data.shape != monthly_cube_30mins_1st_data.shape:
                    print("DIFFERENT SIZEs")
                else:
                
                    # calculate value for second half of hour
                    second_half_of_the_hour_mean_hourly_rainfall_rate_data = 2*cube_1hr_data-monthly_cube_30mins_1st_data
                    # Create a new cube for the second half of the hour (start by copying the first half of hour cube)
                    monthly_cube_30mins_2nd = monthly_cube_30mins_1st.copy()
                    # Set values as calculated
                    monthly_cube_30mins_2nd.data = second_half_of_the_hour_mean_hourly_rainfall_rate_data


                    # ### Edit the times to be 30 mins later
                    # get the times from the first half hour
                    first_half_hour_times = monthly_cube_30mins_1st.coord('time').copy()
                    # add 30 mins
                    second_half_hour_times = first_half_hour_times + 0.5
                    # for the second hald hour cube, remove the time dimension and then re-add the edited one
                    monthly_cube_30mins_2nd.remove_coord('time')
                    monthly_cube_30mins_2nd.add_dim_coord(second_half_hour_times, 0)

                    #######################################################
                    #######################################################
                    ## Join first half hour and second half hour into one cube
                    #######################################################
                    #######################################################
                    # ### Get a list of all the cubes in each of the monthly cubes
                    list_30mins_1st = iris.cube.CubeList(monthly_cube_30mins_1st.slices_over('time'))
                    list_30mins_2nd = iris.cube.CubeList(monthly_cube_30mins_2nd.slices_over('time'))
                    list_30mins = list_30mins_1st +  list_30mins_2nd

                    ### Merge back into one cube
                    monthly_cube_30mins = list_30mins.merge_cube()

                    #######################################################
                    #######################################################
                    ## Save
                    #######################################################
                    #######################################################
                    dir_to_save = f"datadir/UKCP18_every30mins/{resolution}/{em_30mins}/{yrs_range}/"

                    if os.path.isdir(dir_to_save):
                        print("Exists")
                    else:
                        print("Doesn't exist")
                        os.makedirs(dir_to_save)
                    fp_to_save = f"datadir/UKCP18_every30mins/{resolution}/{em_30mins}/{yrs_range}/{em_30mins}a.pr{yr}{month_num}.nc" 
                    print(fp_to_save)
                    iris.save(monthly_cube_30mins, fp_to_save)

NameError: name 'em_matching_dict' is not defined